# Save and load models
- 모델은 훈련하는 중이나 훈련이 끝난 후에 모델을 저장할 수 있다.
- 모델을 중지된 지점부터 다시 훈련할 수 있기 때문에 한 번에 오랫동안 훈련하지 않아도 된다.
- 또 모델을 저장하면 다른 사람과 공유를 할 수 있고 같은 작업을 할 수 있다.
- 기술자들은 모델을 만드는 코드, 모델의 훈련된 가중치 또는 파라미터를 제공하며 연구한 모델과 기법을 공개한다.
- 이런 데이터를 공유하면 다른 사람이 모델의 작동 방식만 이해하고 있다면 새로운 데이터로 모델을 실험할 수 있다.


## 설정
- 필요한 라이브러리를 설치하고 텐서플로우를 import 한다.

In [1]:
pip install -q pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


### 예저 데이터셋 받기
- MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들 것.
- 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 천 개만 사용함.

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 모델 정의

In [4]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 훈련하는 동안 체크 포인트 저장하기
- 훈련 중간과 훈련 마지막에 체크포인트를 자동으로 저장하도록 하는 방법을 많이 사용한다. 
- 다시 훈련하지 않고 모델을 재 사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있다.
- tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백(callback)이다.
- 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개 변수를 제공한다.

### 체크포인트 콜백 만들고 사용하기

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 1s 17ms/step - loss: 1.6511 - sparse_categorical_accuracy: 0.4718 - val_loss: 0.7083 - val_sparse_categorical_accuracy: 0.7770

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4837 - sparse_categorical_accuracy: 0.8708 - val_loss: 0.5304 - val_sparse_categorical_accuracy: 0.8330

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2883 - sparse_categorical_accuracy: 0.9229 - val_loss: 0.4843 - val_sparse_categorical_accuracy: 0.8450

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2110 - sparse_categorical_accuracy: 0.9537 - val_loss: 0.4669 - val_sparse_categorical_accuracy: 0.8600

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1598 - sparse_cat

In [6]:
# 에포크가 종료될 때 마다 업데이트 되는 텐서플로우의 체크포인트 파일의 단일 집합이 생성된다.
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

### 훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가하기
- 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 함.
- 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있음.

In [8]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

# 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준

32/32 - 0s - loss: 2.3247 - sparse_categorical_accuracy: 0.1270
Untrained model, accuracy: 12.70%


In [9]:
# 체크포인트에서 가중치를 로드하고 다시 평가

# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4211 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.30%


### 체크포인트 콜백 매개변수
- 콜백 함수는 몇가지 매개변수를 제공한다.
- 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있다.
- 새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장하는 코드를 짜 볼 것

In [10]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다.
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다.
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다.
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
os.listdir(checkpoint_dir)
# 만들어진 체크포인트를 확인해 보고 마지막 체크포인트를 선택해 봄.

['checkpoint',
 'cp-0005.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

- 텐서플로우는 기본적으로 최근 5개의 체크포인트만 저장한다.

### - 모델을 초기화하고 최근 체크포인트를 로드하여 테스트

In [13]:
# 새로운 모델 객체를 만듭니다.
model = create_model()

# 이전에 저장한 가중치를 로드합니다.
model.load_weights(latest)

# 모델을 재평가합니다.
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5028 - sparse_categorical_accuracy: 0.8740
Restored model, accuracy: 87.40%


---

# < 여기까지 내 발표 내용
## - 이후부터는 민지님 발표 내용

---

### 수동으로 가중치 저장하기

In [14]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5028 - sparse_categorical_accuracy: 0.8740
복원된 모델의 정확도: 87.40%


### 전체 모델 저장하기

### - SaveModel 포맷

In [15]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6145 - sparse_categorical_accuracy: 0.5084
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4529 - sparse_categorical_accuracy: 0.8746
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3382 - sparse_categorical_accuracy: 0.9116
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2044 - sparse_categorical_accuracy: 0.9533
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1791 - sparse_categorical_accuracy: 0.9698
INFO:tensorflow:Assets written to: saved_model/my_model/assets


- SavedModel 포맷은 프로토콜 버퍼 이진 파일과 텐서플로 체크포인트를 담고 있는 디렉토리임.
- 저장된 모델의 디렉토리를 확인 해 봐야 함.

In [16]:
ls saved_model
# my_model 디렉토리 확인

my_model/


In [17]:
ls saved_model/my_model
# assests 폴더, saved_model.pb, variables 폴더

assets/  saved_model.pb  variables/


### 저장된 모델로부터 새로운 케라스 모델을 로드

In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


- 복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있음.
- 이 모델을 평가하고 예측에 사용해 볼 것.

In [19]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4399 - sparse_categorical_accuracy: 0.8510
Restored model, accuracy: 85.10%
(1000, 10)


### HDF5 파일로 저장하기
- 케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공함.

In [20]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6306 - sparse_categorical_accuracy: 0.4846
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4620 - sparse_categorical_accuracy: 0.8888
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3050 - sparse_categorical_accuracy: 0.9061
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1967 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1649 - sparse_categorical_accuracy: 0.9660


In [21]:
#이제 이 파일로부터 모델을 다시 만들어 봄.

# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
#정확도 확인

loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4262 - sparse_categorical_accuracy: 0.8670
복원된 모델의 정확도: 86.70%


# 정리
- 케라스는 모델 구조를 파악하여 저장합니다. 이 방식은 다음과 같은 모든 것을 저장함.
  - 가중치 값
  - 모델 구조
  - 모델의 훈련 설정(컴파일할 때 전달한 값)
  - 옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작할 수 있음)
- 체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)를 저장할 수 없음.
- v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 함.
- 따라서 옵티마이저의 상태를 잃게 됨.